# KPIs and Status Summary
This notebook calculates a number of KPI metrics and a summary of status for test results given the user-specified query and grouping parameters. It ties into the **Test Monitor Service** for retrieving filtered test results, the **Notebook Execution Service** for running outside of Jupyterhub, and **Grafana Dashboards** for displaying results.

The parameters and output use a schema recognized by the NI Plotly Graph Plugin for Grafana.

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service. The SystemLink Test Monitor Client provides access to test result data for processing.

In [ ]:
import copy
import datetime
import pandas as pd
import numpy as np
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

### Parameters
- `results_filter`: Dynamic Linq query filter for test results from the Test Monitor Service
  - Options: Any valid Test Monitor Results Dynamic Linq filter
  - Default: `'startedWithin <= "30.0:0:0"'`
- `results_filter`: Dynamic Linq query filter for test results based on products from the Test Monitor Service
  - Options: Any valid Test Monitor Product Dynamic Linq filter
  - Default: `''`
- `group_by`: The dimension along which to reduce; what each bar in the output graph represents  
  - Options: Day, System, Test Program, Operator, Part Number  
  - Default: `'Day'`

Parameters are also listed in the metadata for the parameters cell, along with their default values. The Notebook Execution services uses that metadata to pass parameters from the Test Monitor Reports page to this notebook. Available `group_by` options are listed in the metadata as well; the Test Monitor Reports page uses these to validate inputs sent to the notebook.

To see the metadata, select the code cell and click the wrench icon in the far left panel.

In [ ]:
results_filter = 'partNumber == "HR-3UTG-AMZN" && startedWithin <= "30.0:0:0"'
products_filter = ''
group_by = 'Day'

### Mapping from grouping options to Test Monitor terminology
Translate the grouping options shown in the Test Monitor Reports page to keywords recognized by the Test Monitor API.

In [ ]:
groups_map = {
    'Day': 'started_at',
    'System': 'system_id',
    'Test Program': 'program_name',
    'Operator': 'operator',
    'Part Number': 'part_number',
    'Workspace': 'workspace'
}
grouping = groups_map[group_by]

### Create Test Monitor client
Establish a connection to SystemLink over HTTP.

In [ ]:
results_api = testmon.ResultsApi()

### Query for results
Query the Test Monitor Service for results matching the `results_filter` parameter.

In [ ]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT)


results = []

response = await results_api.query_results_v2(post_body=results_query)
while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)

results_list = [result.to_dict() for result in results]

### Get group names
Collect the group name for each result based on the `group_by` parameter.

In [ ]:
group_names = []
for result in results_list:
    if grouping in result:
        group_names.append(result[grouping])

### Create pandas dataframe
Put the data into a dataframe whose columns are serial number, status, start time, and group name. Sort and group the dataframe to get the first test run for each unique serial number.

In [ ]:
formatted_results = {
    'id': [result['id'] for result in results_list],
    'serial_number': [result['serial_number'] for result in results_list],
    'status': [result['status']['status_type'] for result in results_list],
    'started_at': [result['started_at'] for result in results_list],
    'system_id': [result['system_id'] for result in results_list],
    'total_time_in_seconds': [result['total_time_in_seconds'] for result in results_list],
    grouping: group_names
}

df_results = pd.DataFrame.from_dict(formatted_results)

if grouping == 'started_at':
    sorting_list = ['serial_number', 'started_at']
    grouping_list = ['serial_number']

else:
    sorting_list = [grouping, 'serial_number', 'started_at']
    grouping_list = [grouping, 'serial_number']

df_results = df_results.sort_values(by=sorting_list)
df_results_unique = df_results.groupby(grouping_list).first().reset_index()

### Handle grouping by day
If the grouping is by day, the group name is the date and time when the test started in UTC. To group all test results from a single day together, convert to server time and remove time information from the group name.

In [ ]:
df_results_copy = copy.copy(df_results_unique)
df_results_copy.fillna(value='', inplace=True)

if grouping == 'started_at':
    truncated_times = []
    for val in df_results_copy[grouping]:
        local_time = val.astimezone(tz.tzlocal())
        truncated_times.append(str(datetime.date(local_time.year, local_time.month, local_time.day)))
    df_results_copy[grouping] = truncated_times

### Aggregate results into groups
Aggregate the data for each unique group and status.

*See documentation for [size](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.size.html) and [unstack](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) here.*

In [ ]:
df_grouped_status = df_results_copy.groupby([grouping, 'status']).size().unstack(fill_value=0)
if 'PASSED' not in df_grouped_status:
    df_grouped_status['PASSED'] = 0
if 'FAILED' not in df_grouped_status:
    df_grouped_status['FAILED'] = 0
if 'ERRORED' not in df_grouped_status:
    df_grouped_status['ERRORED'] = 0
if 'TERMINATED' not in df_grouped_status:
    df_grouped_status['TERMINATED'] = 0
    
df_grouped_status

### KPI calculations

First Pass Yield - Divide the number of passed tests by the total number of tests.

In [ ]:
df_first_pass_yield = pd.DataFrame(100 * df_grouped_status['PASSED'] / (df_grouped_status['FAILED'] + df_grouped_status['ERRORED'] + df_grouped_status['PASSED']))

df_kpis = df_first_pass_yield.reset_index().set_axis([grouping, 'yield'], axis=1)

Throughput - Count the number of tests per group

In [ ]:
df_throughput = df_results_copy.groupby(grouping).agg({'id': 'count'})
df_throughput = df_throughput.reset_index().set_axis([grouping, 'throughput'], axis=1)

df_kpis = df_kpis.merge(df_throughput, on=grouping)

Failure Rate - Divide the number of failed and errored tests by the total number of tests.

In [ ]:
df_fail_rate = pd.DataFrame(100 * (df_grouped_status['FAILED'] + df_grouped_status['ERRORED'])/ (df_grouped_status['FAILED'] + df_grouped_status['ERRORED'] + df_grouped_status['PASSED']))

if df_fail_rate.empty:
    df_fail_rate = pd.DataFrame(columns=[grouping, 'fail_rate'])
    df_kpis['fail_rate'] = ''
else:
    df_fail_rate = df_fail_rate.reset_index().set_axis([grouping, 'fail_rate'], axis=1)
    df_kpis = df_kpis.merge(df_fail_rate, on=grouping)


System Utilization - Divide how much time testing occurred by the time available for testing

In [ ]:
system_availability_by_day = {
    'MONDAY': 8,
    'TUESDAY': 8,
    'WEDNESDAY': 8,
    'THURSDAY': 8,
    'FRIDAY': 8,
    'SATURDAY': 0,
    'SUNDAY': 0
}

number_of_unique_systems = df_results['system_id'].nunique()

dates = []
for val in df_results['started_at']:
    dates.append(val)
if dates:
    dates.sort()
    start_date = datetime.datetime(dates[0].year, dates[0].month, dates[0].day)
    end_date = datetime.datetime(dates[-1].year, dates[-1].month, dates[-1].day) + datetime.timedelta(days=1)

group_info = {
    grouping: [],
    'utilization': []
}

for group in df_results_copy[grouping].unique():
    group_df = df_results_copy[df_results_copy[grouping] == group]
    group_info[grouping].append(group)
    current_date = start_date
    available_in_seconds = 0
    while current_date < end_date:
        available_in_seconds += system_availability_by_day[current_date.strftime('%A').upper()] * 60 * 60
        current_date += datetime.timedelta(days=1)
    group_info['utilization'].append(100 * (group_df['total_time_in_seconds'].sum() / available_in_seconds))

df_system_utilization = pd.DataFrame.from_dict(group_info)

if grouping == 'started_at':
    df_system_utilization['utilization'] = df_system_utilization['utilization'] / number_of_unique_systems
else:
    df_system_utilization.sort_values(by=['utilization'], ascending=True, inplace=True)
    
df_kpis = df_kpis.merge(df_system_utilization, on=grouping)

Cleanup

In [ ]:
if grouping == 'started_at':
    df_kpis['started_at'] = pd.to_datetime(df_kpis['started_at'])
else:
    df_kpis.sort_values(by=['yield'], ascending=True, inplace=True)

### Convert the dataframe to the SystemLink reports output format
The result format for a SystemLink report consists of a list of output objects as defined below:
- `type`: The type of the output. Accepted values are 'data_frame' and 'scalar'.
- `id`: Corresponds to the id specified in the 'output' metadata. Used for returning multiple outputs with the 'V2' report format.
- `data`: A dict representing the 'data_frame' type output data.
    - `columns`: A list of dicts containing the names and data type for each column in the dataframe.
    - `values`: A list of lists containing the dataframe values. The sublists are ordered according to the 'columns' configuration.
- `value`: The value returned for the 'scalar' output type.
- `config`: The configurations for the given output.
    - `title`: The output title.
    - `graph`: The graph configurations.
        - `axis_labels`: The x-axis label and y-axis label.
        - `plots`: A list of plots to display mapped from the dataframe's columns, along with configuration options.
            - `x`: The dataframe column corresponding to the x-axis values.
            - `y`: The dataframe column corresponding to the y-axis values.
            - `style`: The plot's style. Accepted values are ['LINE', 'BAR', 'SCATTER'].
            - `color`: The plot's color. Accepted formats are ['blue', '#0000ff', 'rbg(0,0,255)'].
            - `label`: The plot's name, to be shown in a plot legend. 
            - `secondary_y`: Whether or not to display this plot on a second y-axis.
            - `group_by`: A list of columns in the dataframe on which to group data, e.g. to color individual points.
        - `orientation`: 'HORIZONTAL' or 'VERTICAL'.
        - `stacked`: Whether or not to display the plots stacked on top of each other.

For this report, there are two outputs.
The first is a dataframe with five columns representing KPIs. For a grouping of 'Day', the first column contains ISO-8601 date strings. For any other grouping option, the first column contains categorical string values. The second column contains numerical values representing the yield percentages, etc.

| started_at                 | yield         | throughput | fail_rate         | utilization|
|----------------------------|---------------|------------|-------------------|------------|
| '2020-09-29'               | 84.87         | 456        | 15.13             | 30.25      | 
| '2020-09-30'               | 65.58         | 154        | 37.85             | 10.5       |
| '2020-10-01'               | 86.43         | 258        | 8.53              | 25.75      |

The second is a dataframe with five columns representing status summary data.  For a grouping of 'Day', the first column contains ISO-8601 date strings. For any other grouping option, the first column contains categorical string values. The second column contains the count of passed tests, etc.

| started_at                 | PASSED        | FAILED     | ERRORED           | TERMINATED |
|----------------------------|---------------|------------|-------------------|------------|
| '2020-09-29'               | 387           | 54         | 15                | 0          | 
| '2020-09-30'               | 101           | 27         | 26                | 0          |
| '2020-10-01'               | 245           | 15         | 7                 | 13         |


The graph configuration does not specify any configuration or plot information since Grafana users will provide this information in their dashboard configuration instead. We use Pandas to convert the dataframe built in the previous cells into a tabular format and then return that with the result object.

In [ ]:
df_dict_kpis = {
    'columns': pd.io.json.build_table_schema(df_kpis, index=False)['fields'],
    'values': df_kpis.values.tolist(),
}

kpis_graph = {
    'type': 'data_frame',
    'id': 'kpis_graph',
    'data': df_dict_kpis
}

df_grouped_status.reset_index(inplace=True)
df_dict_status_count = {
    'columns': pd.io.json.build_table_schema(df_grouped_status, index=False)['fields'],
    'values': df_grouped_status.values.tolist(),
}

status_count_graph = {
    'type': 'data_frame',
    'id': 'status_count_graph',
    'data': df_dict_status_count
}

result = [kpis_graph, status_count_graph]

### Record results with Scrapbook

In [ ]:
sb.glue('result', result)